In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
file = "data/starwars/starwars-full-interactions-allCharacters-merged.json"

In [5]:
import json
import os

# Get the absolute path to the file
base_dir = os.path.dirname(os.path.abspath("/Users/august/Desktop/AMAS2026/AMAS_project_2026/notebooks/august_initial.ipynb"))
file_path = os.path.join(base_dir, "../data/starwars/starwars-full-interactions-allCharacters-merged.json")

# Load the JSON file
with open(file_path, 'r') as f:
    data = json.load(f)

# Extract all names from the "nodes" category
names = [node['name'] for node in data['nodes']]

print(f"Total nodes: {len(names)}")
print(f"Names: {names}")

Total nodes: 111
Names: ['DARTH VADER', 'R2-D2', 'CHEWBACCA', 'BB-8', 'QUI-GON', 'NUTE GUNRAY', 'PK-4', 'TC-14', 'OBI-WAN', 'DOFINE', 'RUNE', 'TEY HOW', 'EMPEROR', 'CAPTAIN PANAKA', 'SIO BIBBLE', 'JAR JAR', 'TARPALS', 'BOSS NASS', 'PADME', 'RIC OLIE', 'WATTO', 'SEBULBA', 'JIRA', 'SHMI', 'C-3PO', 'DARTH MAUL', 'KITSTER', 'WALD', 'FODE/BEED', 'JABBA', 'GREEDO', 'VALORUM', 'MACE WINDU', 'KI-ADI-MUNDI', 'YODA', 'RABE', 'BAIL ORGANA', 'GENERAL CEEL', 'BRAVO TWO', 'BRAVO THREE', 'CAPTAIN TYPHO', 'SENATOR ASK AAK', 'ORN FREE TAA', 'SOLA', 'JOBAL', 'RUWEE', 'TAUN WE', 'LAMA SU', 'BOBA FETT', 'JANGO FETT', 'OWEN', 'BERU', 'CLIEGG', 'COUNT DOOKU', 'SUN RIT', 'POGGLE', 'PLO KOON', 'ODD BALL', 'GENERAL GRIEVOUS', 'FANG ZAR', 'MON MOTHMA', 'GIDDEAN DANU', 'CLONE COMMANDER GREE', 'CLONE COMMANDER CODY', 'TION MEDON', 'CAPTAIN ANTILLES', 'LUKE', 'CAMIE', 'BIGGS', 'LEIA', 'MOTTI', 'TARKIN', 'HAN', 'DODONNA', 'GOLD LEADER', 'WEDGE', 'RED LEADER', 'RED TEN', 'GOLD FIVE', 'RIEEKAN', 'DERLIN', 'ZEV', 'PIE